## Benchmark Training for evaluating a new network

This will test the new network on two tasks

1. Single object performance (caterpillar, baymax, starbot dataset) with no cross-object loss
2. Class consistent performance on shoes dataset

Set the `name` variable below to a distinctive name for this network architecture

In [ ]:
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
from dense_correspondence.training.training import *
import sys
import logging
import copy
# utils.set_default_cuda_visible_devices()
utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
logging.basicConfig(level=logging.INFO)

In [ ]:
train_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'training.yaml')

default_train_config = utils.getDictFromYamlFilename(train_config_file)

##### CHANGE THIS TO YOUR NETWORK NAME ##############
name = "standard"

logging_dir = os.path.join("trained_models/benchmark", name) # this is where trained models will be saved
num_iterations = 3500
num_image_pairs = 100


default_train_config['training']['num_iterations'] = num_iterations
default_train_config['training']['logging_dir'] = logging_dir



TRAIN = True
EVALUATE = True
EVALUATE_KEYPOINTS = True


## Train & Eval on Caterpillar-Baymax-Starbot dataset

In [ ]:
dataset_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'caterpillar_baymax_starbot_all_front_single_only.yaml')
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)
dataset = SpartanDataset(config=dataset_config)

# make any necessary tweaks to your default_train_config
train_config = copy.copy(default_train_config)


train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
train_config = utils.getDictFromYamlFilename(train_config_file)
network_name = "cbs_%d" %(train_config['dense_correspondence_network']['descriptor_dimension'])
train._config["training"]["logging_dir"] = logging_dir
train._config["training"]["logging_dir_name"] = network_name


if TRAIN:
    train.run()

model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_data_relative_path_to_absolute_path(model_folder)

if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)

## Train & Eval class consistent network on shoes dataset

In [ ]:
dataset_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'shoe_train_all_shoes.yaml')
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)
dataset = SpartanDataset(config=dataset_config)

keypoint_data_file = "evaluation_labeled_data/keypoints/shoe_keypoints.yaml"

# make any necessary tweaks to your default_train_config
train_config = copy.copy(default_train_config)


train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
train_config = utils.getDictFromYamlFilename(train_config_file)
network_name = "shoes_consistent_%d" %(train_config['dense_correspondence_network']['descriptor_dimension'])
train._config["training"]["logging_dir"] = logging_dir
train._config["training"]["logging_dir_name"] = network_name


if TRAIN:
    train.run()

model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_data_relative_path_to_absolute_path(model_folder)

if EVALUATE:
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)

if EVALUATE_KEYPOINTS:
    DCE.run_cross_instance_keypoint_evaluation_on_network(model_folder, keypoint_data_file)